# Discrete Event Simulation

This notebook contains code examples referring to the Discrete Event Simulation chapter of "Applied Mathematics with Open-Source Software: Operational Research Problems
with Python and R".

First we define a function that gives a Network object, containing the structure of the repair shop.

In [1]:
import ciw


def build_network_object(
    num_inspectors=1,
    num_repairers=2,
):
    """Returns a Network object that defines the repair shop.

    Args:
        num_inspectors: a positive integer (default: 1)
        num_repairers: a positive integer (default: 2)

    Returns:
        a Ciw network object
    """
    arrival_rate = 15
    inspection_rate = 20
    repair_rate = 10
    prob_need_repair = 0.8
    N = ciw.create_network(
        arrival_distributions=[
            ciw.dists.Exponential(arrival_rate),
            ciw.dists.NoArrivals(),
        ],
        service_distributions=[
            ciw.dists.Exponential(inspection_rate),
            ciw.dists.Exponential(repair_rate),
        ],
        number_of_servers=[num_inspectors, num_repairers],
        routing=[[0.0, prob_need_repair], [0.0, 0.0]],
    )
    return N

We can see information such as number of nodes in the network:

In [2]:
N = build_network_object()
print(N.number_of_nodes)

2


Then we define a function that runs one trial of the simulation.

In [3]:
def run_simulation(network, seed=0):
    """Builds a simulation object and runs it for 8 time units.

    Args:
        network: a Ciw network object
        seed: a float (default: 0)

    Returns:
        a Ciw simulation object after a run of the simulation
    """
    max_time = 8
    ciw.seed(seed)
    Q = ciw.Simulation(network)
    Q.simulate_until_max_time(max_time)
    return Q

From one trial we can obtain the proportion of bicycles taking over half an hour:

In [4]:
import pandas as pd


def get_proportion(Q):
    """Returns the proportion of bicycles spending over a given
    limit at the repair shop.

    Args:
        Q: a Ciw simulation object after a run of the
           simulation

    Returns:
        a real
    """
    limit = 0.5
    inds = Q.nodes[-1].all_individuals
    recs = pd.DataFrame(
        dr for ind in inds for dr in ind.data_records
    )
    recs["total_time"] = recs["exit_date"] - recs["arrival_date"]
    total_times = recs.groupby("id_number")["total_time"].sum()
    return (total_times > limit).mean()

Putting all this together for one trial

In [5]:
N = build_network_object()
Q = run_simulation(N)
p = get_proportion(Q)
print(round(p, 6))

0.261261


A function to find the average proportion over a number of trials

In [6]:
def get_average_proportion(num_inspectors=1, num_repairers=2):
    """Returns the average proportion of bicycles spending over a
    given limit at the repair shop.

    Args:
        num_inspectors: a positive integer (default: 1)
        num_repairers: a positive integer (default: 2)

    Returns:
        a real
    """
    num_trials = 100
    N = build_network_object(
        num_inspectors=num_inspectors,
        num_repairers=num_repairers,
    )
    proportions = []
    for trial in range(num_trials):
        Q = run_simulation(N, seed=trial)
        proportion = get_proportion(Q=Q)
        proportions.append(proportion)
    return sum(proportions) / num_trials


The proportion with current staff:

In [7]:
p = get_average_proportion(num_inspectors=1, num_repairers=2)
print(round(p, 6))

0.159354


The proportion with an extra inspector:

In [8]:
p = get_average_proportion(num_inspectors=2, num_repairers=2)
print(round(p, 6))

0.038477


The proportion with an extra repairer:

In [9]:
p = get_average_proportion(num_inspectors=1, num_repairers=3)
print(round(p, 6))

0.103591
